# **GitHub configuration**

## **Configuring the account**

In [ ]:
# identifying into github
import subprocess, gc, os, torch
from dotenv import load_dotenv

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
gc.collect()
torch.cuda.empty_cache()

!git config --global user.name "Nuzz23"
!git config --global user.email "nunzio.licalzi9@gmail.com"
output = subprocess.check_output("git config --global --list", shell=True).decode('utf-8').split()

# check if correctly identified
assert len(output) >= 2, "Wrong length"
assert output[0].split('=')[-1] == 'Nuzz23', 'wrong user name'
assert output[1].split('=')[-1] == 'nunzio.licalzi9@gmail.com', 'wrong email'
del output

load_dotenv()

## **Cloning the repository**

In [ ]:
# if the cloned repository already exists delete it
%cd /content
%rm -rf sample_data
if "MLDL_SemanticSegmentation" in subprocess.check_output("ls", shell=True).decode("utf-8").strip():
  !rm -rf MLDL_SemanticSegmentation

/content


In [ ]:
# clone the repository via token
!git clone --quiet "{os.getenv('GITHUB_TOKEN')}"

# check if cloned correctly
assert "MLDL_SemanticSegmentation" in subprocess.check_output("ls", shell=True).decode("utf-8").strip(), "Not cloned correctly"

# **installing and importing the libraries**

In [ ]:
# installing the required libraries
%cd MLDL_SemanticSegmentation
%pip install -q -r requirements.txt

[Errno 2] No such file or directory: 'MLDL_SemanticSegmentation'
/content/MLDL_SemanticSegmentation


### **Defining custom imports**

In [ ]:
from stats import countFLOPS, latency
from train.trainDeepLabV2 import initModelDeepLabV2
from datasets.downloader import Downloader

# **DATASET**

## **Downloading the dataset**

In [ ]:
if not Downloader().downloadCityScapes():
  raise FileNotFoundError("CityScapes dataset not found")

Error downloading CityScapes dataset: Command '['gdown', '--quiet', '1uINto6zVV0VY9h380Pnprt_PR0izLfzW', '-O', 'CityScapes.zip']' returned non-zero exit status 1. with id 1uINto6zVV0VY9h380Pnprt_PR0izLfzW
Error downloading CityScapes dataset: Command '['gdown', '--quiet', '1O8VGxrdRgRCt8lw36SXFNPIvfYnDZvOs', '-O', 'CityScapes.zip']' returned non-zero exit status 1. with id 1O8VGxrdRgRCt8lw36SXFNPIvfYnDZvOs


## **Weights of the pretrained model**

In [ ]:
if not Downloader().downloadWeightsDeepLabV2():
  raise FileNotFoundError("DeepLabV2 weights not found")

# **Model**

## **Main Function**

In [ ]:
WIDTH, HEIGHT = 1024, 512

model = initModelDeepLabV2(width=WIDTH, height=HEIGHT, pushWeights=False, enablePrintVal=True, restartTraining=True)

In [ ]:
print('flops', countFLOPS(model, width=WIDTH, height=HEIGHT))

print('latencyMean, latencyStd, fpMean, fpsStd', *latency(model, width=WIDTH, height=HEIGHT))

print(f"Tot number of parameters: {sum(p.numel() for p in model.parameters())}")

flops 374664739584
latencyMean, latencyStd, fpMean, fpsStd 261.788982629776 11.064570867830977 3.8798695890840054 1.5657381373703696
Numero totale di parametri: 43901068
